In [1]:
pip install soundfile numpy sklearn pyaudio


Note: you may need to restart the kernel to use updated packages.


In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Features (mfcc=Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound, 
#chroma=Pertains to the 12 different pitch classes, 
#mel= Mel Spectrogram Frequency) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [5]:
#emotions defined and required emotions that are to be compared specified
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
#Loading and extracting features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\aryan\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):#Path of data set on users computer 
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [29]:
# Spliting the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)#here data percentage can be changed 
# Shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))
#the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(120, 40)
Features extracted: 180


In [30]:
#Initializeing the Multi Layer Perceptron Classifier
#using mlpclassifier which connects to a neural network 
model=MLPClassifier(alpha=0.01, batch_size=16, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [31]:
#Training the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=16, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [32]:
#Prediction for the test set
y_pred=model.predict(x_test)

In [33]:
#Calculating the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#Printing the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.00%


In [34]:
#acuuracy increased as more refined data set used 
# model helpful in medical field with patients with autism or other neurological issues that negate 
# the ability of the person to recognise emotions 
# as further development done a user friendly and easy to use model can be generated .
#thank you